# Covod19 Chatbot Demo

In [1]:
from sentence_transformers import SentenceTransformer, util
import csv
import pandas as pd

In [2]:
base_model = 'bert-base-nli-mean-tokens-eps1-batch16-lr2e-05'
model = SentenceTransformer('./model/' + base_model)

In [3]:
covid19_QA_data = csv.reader(open('data/WHO/covid19_QA_data.csv'))

WHO_covid19_QA = {}
for row in covid19_QA_data:
    question = row[1]
    WHO_covid19_QA[question] = row[2]
    
# Extract All WHO Covid19 Questions
sentences1 = pd.read_csv('data/WHO/covid19_QA_data.csv')['question'].tolist()

def chatbot(user_query):
    
    # Chatbot get User Query
    sentences2 = [user_query] * len(sentences1)

    # Compute Embeddings for Sentences
    embeddings1 = model.encode(sentences1, convert_to_tensor=True)
    embeddings2 = model.encode(sentences2, convert_to_tensor=True)

    # Compute Cosine-Similarits
    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

    # Output Similarity Score
    df = pd.DataFrame(columns=['WHO Question','Similarity Score'])
    for i in range(len(sentences1)):
        df.loc[i] = [sentences1[i], cosine_scores[i][i].item()]
    df = df.sort_values('Similarity Score', ascending=False)
    
    # Get Answer by mapping WHO_covid19_QA Dict
    q = df.iloc[0]['WHO Question']
    a = WHO_covid19_QA[q]
    top5 = df[:5]
    return q, a, top5

In [4]:
user_query = "Introduction to Coronavirus"
q, a, top5 = chatbot(user_query)

print("Query: {}".format(user_query))
print("Chatbot Answer: {}".format(a))

Query: Introduction to Coronavirus
Chatbot Answer: Coronaviruses are a large family of viruses which may cause illness in animals or humans.  In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS). The most recently discovered coronavirus causes coronavirus disease COVID-19.
